<h2><center> EEG - N. 9 </center></h2>

<h3><center> MANU 465 101 </center></h3>

[Set phase](#Set-phase)

 - [Import libraries](#Import-libraries)
 - [Setting path](#Setting-path)

[Features generation](#Features-generation)


[Import procedure](#Import-procedure)
  - [Lists initialization](#Lists-initialization)
  - [Final dataset creation](#Final-dataset-creation)
       
[Conclusions](#Conclusions)
       

# Set phase

### Import libraries

In [166]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import scipy.stats as stats

### Setting path

In [167]:
delimiter = "\\" # for windows
#delimiter = "/"  # for os system
path = r"C:\Users\chies\OneDrive\Desktop\MANU-465-EEG4-main\Data Collection\Unprocessed Dataset\Right Hand Dominant"

In [168]:
os.chdir(path)

Get all the filenames in the folder indicated by the previous path

In [169]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [172]:
#all_filenames

In [170]:
len(all_filenames)

321

# Features generation

In order to condensate each csv file in only one row, we need to use summary metrics. 

For each wave, which corresponds to 4 columns in each file, we want one value of mean, one value of std, and so on. Then all these values will be stored in a final dataset in a unique row that represents one csv file.

In [171]:
from scipy import stats 
def mean(x):
    return np.mean(x, axis=0)
def std(x):
    return np.std(x, axis=0)
def ptp(x):
    return np.ptp(x, axis=0)
def var(x):
    return np.var(x, axis=0)
def minim(x):
    return np.min(x, axis=0)
def maxim(x):
    return np.max(x, axis=0)
def argminim(x):
    return np.argmin(x, axis=0)
def argmaxim(x):
    return np.argmax(x, axis=0)
def rms(x):
    return np.sqrt(np.mean(x**2, axis=0))
def abs_diff_signal(x):
    np.sum(np.abs(np.diff(x,axis=0)),axis=0)
def skewness(x):
    return stats.skew(x,axis=0)
def kurtosis(x):
    return stats.kurtosis(x,axis=0)

def concatenate_features(x):
    '''''''''
    this function apply several functions defined above.
    It takes as input a numpy array.
    It outputs a vector with the value of each function: mean, std, ...
    '''''''''
    return mean(x),std(x),ptp(x),var(x),minim(x),maxim(x),argminim(x),argmaxim(x),rms(x),abs_diff_signal(x),skewness(x),kurtosis(x)

# Import procedure

## Lists initialization

Creation of useful lists and the final dataset initialization.

The final dataset should has some columns related to information about the patient (Gender, Dominance hand, ID Number, Test, ...), 12 columns for each wave (for each wave we have mean, std, ... (in total 12 new features)). 

Since we tested each patient 4 times, the dimension of the final dataset will be:
- number rows = 4 * number of patients = number of csv files
- number columns = 12 * 5 (features * waves numbers) + fixed qualities (persona data)

In [173]:
waves = ["Delta", "Theta", "Alpha", "Beta", "Gamma"] # names of waves we are interested in

In [174]:
test_list = []
dominance_list = []
english_list = []
gender_list = []
participant_list = []

# dictionary to store all the values for one dataframe
final_dic = {}

## Final dataset creation

In [175]:
for name in all_filenames[0:30]: # file 6 contains string number
    
    df = pd.read_csv(path + delimiter + name)

    df = df.drop(["Elements"], axis=1)
    df = df.dropna() # drop Nan values
    df = df.reset_index(drop=True) # restart index
    
    # Add to lists the values related to patient profile
    test_list.append(df["Test"][0])
    dominance_list.append(df["Dominance"][0])
    english_list.append(df["English"][0])
    gender_list.append(df["Gender"][0])
    participant_list.append(df["Participant"][0])

    # create empty list to collect mean, std, var, ... for a fixed wave
    gen_features = {}
    
    #print(name)
    
    for wave in waves: # for each waves (ALpha, Beta, Gamma, Delta, Theta)
        
        # create empty list to collect all the data in the four columns for a fixed wave
        all_values = []
        
        for col in df.columns: # for each column
            if col.split("_")[0] == wave: # if the wave is in the column name, then:
                
                # Marti code
                # to clean data, we delete columns in which the values is equal to the one 4 time points before
                for i in range(4,len(df[col])):
                    if not df[col][i] == df[col][i-4]:
                        all_values.append(df[col][i])
                #if len(all_values)!=0: # if the values are not constant, we want to add also the first 4 values
                #    for i in range(4):
                #        all_values.append(df[col][i])
                for i in range(4):
                    all_values.append(df[col][i])  
                    
        # add a list with the new features associated to the name of the wave in a dictionary
        gen_features[wave] = list( concatenate_features(np.array(all_values)) )
    
    final_dic[name] = gen_features
         
#print(final_dic)

In [176]:
# create an empty dataframe
final_df = pd.DataFrame()

# assign to the column 'Test' of the final df all the values which are in the test_list
final_df["Test"] = test_list
final_df["Dominance"] = dominance_list
final_df["English"] = english_list
final_df["Gender"] = gender_list
final_df["Participant"] = participant_list

functions = ["mean", "std", "ptp","var","minim","maxim","argminim","argmaxim","rms","abs_diff_signal","skewness","kurtosis"] 

for i in range(len(all_filenames[0:30])): # i indicates the row (index for each file) 
    name = all_filenames[i]
    for wave in waves: # for each wave
        for j in range(12): 
            # at row i, and column specified by the name of the wave and features
            final_df.at[i, wave + "_" + functions[j]] = final_dic[name][wave][j]       
            
final_df

,Test,Dominance,English,Gender,Participant,Delta_mean,Delta_std,Delta_ptp,Delta_var,Delta_minim,...,Gamma_ptp,Gamma_var,Gamma_minim,Gamma_maxim,Gamma_argminim,Gamma_argmaxim,Gamma_rms,Gamma_abs_diff_signal,Gamma_skewness,Gamma_kurtosis
0,LHC,Right,Yes,Female,105.0,0.114491,1.162867,5.343660,1.352260,-3.927080,...,3.862173,0.956092,-2.915086,0.947087,71.0,7.0,1.015837,NaN,-2.457083,4.627125
1,LHC,Right,Yes,Male,106.0,0.567189,0.504123,2.265507,0.254140,-0.570804,...,1.762350,0.397326,-0.568534,1.193816,47.0,91.0,0.761039,NaN,-0.310182,-1.689339
2,LHC,Right,Yes,Male,107.0,0.368630,0.225931,0.984091,0.051045,-0.024204,...,1.266026,0.078881,0.000000,1.266026,46.0,32.0,0.761709,NaN,-0.811450,1.040798
3,LHC,Right,Yes,Male,108.0,0.528756,0.366004,1.930394,0.133959,-0.617579,...,1.896629,0.414249,-0.364225,1.532404,24.0,89.0,0.795498,NaN,0.030070,-1.779677
4,LHC,Right,Yes,Male,109.0,0.566548,0.357839,1.321343,0.128049,-0.040740,...,1.150122,0.052284,0.000000,1.150122,50.0,18.0,0.624397,NaN,-0.204306,0.418398
5,LHC,Right,Yes,Male,110.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,NaN,0.000000,-3.000000
6,LHC,Right,Yes,Male,111.0,0.182188,0.335858,1.634503,0.112801,-0.550533,...,1.593745,0.217813,-0.749782,0.843962,56.0,152.0,0.487470,NaN,-0.308617,-1.360148
7,LHC,Right,Yes,Male,112.0,0.710636,0.275964,1.473381,0.076156,-0.003454,...,1.132306,0.069292,-0.331337,0.800969,93.0,72.0,0.506851,NaN,-1.460425,1.176831
8,LHC,Right,Yes,Female,113.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,NaN,0.000000,-3.000000
9,LHC,Right,No,Male,114.0,0.409111,0.358305,1.517782,0.128382,-0.324053,...,1.368946,0.112732,-0.332126,1.036820,78.0,32.0,0.461349,NaN,-0.097610,-0.928321


Merge this with also the other folder files